## Protein Processing Filteration

In [22]:
import pandas as pd
import numpy as np 
import os


In [60]:
    # Define the path to the folder
folder_path= 'DATASETS\DATASET2_STOMACH'
rootFolder = 'Dataset2 (Stomach)'
controlFileName='IPIgGFS2.xls'
experiFileName='IPMWFS2.xls'

### Files to read|

In [61]:


genderData = controlFileName[5:-4]
# Define the root folder name


# Sub Folder
part1Folder = 'Part 1'
part2Folder = 'Part 2'
finalScreenData = 'Final Screen Data'

part1ExclusiveFileName= f'Exclusive{genderData}.xlsx'
part1SimilarFileName=f'Similar{genderData}.xlsx'

# Define the full path to the root folder
rootFolder_path = os.path.join(os.getcwd(), rootFolder)

# Create the root folder
os.makedirs(rootFolder_path, exist_ok=True)

# Create subfolders inside the root folder
os.makedirs(os.path.join(rootFolder_path, part1Folder), exist_ok=True)
os.makedirs(os.path.join(rootFolder_path, part2Folder), exist_ok=True)
os.makedirs(os.path.join(rootFolder_path, finalScreenData), exist_ok=True)



# List all files in the folder
files = os.listdir(folder_path)

# Assuming you want to read the first file in the folder
if files:
#     file_to_read = files[0]

    CONTROL_FILE= os.path.join(folder_path,controlFileName )
    EXPERIMENT_FILE = os.path.join(folder_path, experiFileName)

    CONTROL_DATA = pd.read_excel(CONTROL_FILE)
    EXPERIMENT_DATA = pd.read_excel(EXPERIMENT_FILE)
    print(CONTROL_DATA.head())  # Displaying the first few rows of the DataFrame
    print(EXPERIMENT_DATA.head())


  accession       entry  coverage  proteinProbability  totalPeptides  \
0    P01837  IGKC_MOUSE     36.45              1.0000              3   
1    P60710  ACTB_MOUSE     63.47              1.0000             21   
2    Q9CQP0  RM33_MOUSE     13.85              0.9987              1   
3    P01942   HBA_MOUSE     42.96              1.0000              6   
4    Q9CR36  GKN1_MOUSE     29.35              1.0000              4   

   uniquePeptides  razorPeptides  totalSpectralCount  uniqueSpectralCount  \
0               3              3                  20                   20   
1               1             21                  93                    2   
2               1              1                   1                    1   
3               5              6                  25                   24   
4               4              4                  31                   31   

   razorSpectralCount  ...  uniqueIntensity  razorIntensity  \
0                  20  ...     6.910275e+

## Primary Screening For exclusive proteins

In [62]:
# Extract the 'entry' column from each DataFrame
control_subset = CONTROL_DATA[['entry', 'coverage', 'totalPeptides']]
experiment_subset = EXPERIMENT_DATA[['entry', 'coverage', 'totalPeptides']]

#Difference
# Find proteins present in the experiment dataset but not in the control dataset
missing_proteins = experiment_subset[~experiment_subset['entry'].isin(control_subset['entry'])]

# Total Peptides >= 10 in missing proteins 
tPCount = missing_proteins[missing_proteins['totalPeptides'] >=10]
missing_proteins = tPCount.sort_values(by='coverage',ascending=False)
print(missing_proteins)



            entry  coverage  totalPeptides
188   TBB2A_MOUSE     39.78             16
189   TBB2B_MOUSE     39.78             16
227   GRM2B_MOUSE     31.24             10
202    TBB6_MOUSE     27.29             12
482   NCOA5_MOUSE     25.04             13
620   ASTRB_MOUSE     19.65             12
187   K2C6A_MOUSE     16.46             10
1319  MYO1A_MOUSE     14.96             17
1378   ERN2_MOUSE     13.39             10
591   MYH10_MOUSE      8.45             16
2129  AKAP9_MOUSE      8.40             29
1921   PYR1_MOUSE      8.36             17
2205  ITPR3_MOUSE      4.79             13


In [63]:
# Assuming 'missing_proteins' is your DataFrame

def saveInFolder(dataFrame,file_name,partName):
# Define the folder path relative to the root directory
    rootF = os.getcwd() # Replace '/path/to/root/folder' with the actual root folder path
    folder_path = os.path.join(rootF, rootFolder , partName)

    # Define the full file path
    file_path = os.path.join(folder_path, file_name)

    # Ensure that the folder exists, if not create it
    os.makedirs(folder_path, exist_ok=True)

    # Save the DataFrame to an Excel file in the specified folder
    dataFrame.to_excel(file_path, index=False)

    print(f"{file_name} is saved to:", file_path)
    
def saveInSheet(dataFrame,file_name,partName,sheetName):
    rootF = os.getcwd() # Replace '/path/to/root/folder' with the actual root folder path
    folder_path = os.path.join(rootF,rootFolder , partName)

    # Define the full file path
    file_path = os.path.join(folder_path, file_name)

    # Ensure that the folder exists, if not create it
    os.makedirs(folder_path, exist_ok=True)

    # Save the DataFrame to an Excel file in the specified folder
    
    # Check if the file exists before attempting to append data
    if not os.path.exists(file_path):
        # If the file doesn't exist, create it with a new ExcelWriter
        with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
            dataFrame.to_excel(writer, sheet_name=sheetName, index=False)
    else:
        # If the file already exists, open it with mode='a' to append data
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
            # Save the DataFrame to the specified sheet
            dataFrame.to_excel(writer, sheet_name=sheetName, index=False)

    print(f"{file_name} is saved.")


In [64]:
saveInFolder(missing_proteins,part1ExclusiveFileName,part1Folder)


ExclusiveFS2.xlsx is saved to: C:\Users\Jawwad\Desktop\Zaid Bhai Research\Dataset2 (Stomach)\Part 1\ExclusiveFS2.xlsx


## Primary Screening For Similarity 

In [65]:
#Similarity 
# Finding Proteins present in Both Data Set.
# Merge datasets based on the 'entry' column to get common entries
common_entries = pd.merge(CONTROL_DATA, EXPERIMENT_DATA, on='entry', suffixes=('_control', '_experiment'))

# Extract the 'totalPeptide' columns for common entries
common_entries_total_peptide = common_entries[['entry', 'totalPeptides_experiment','totalPeptides_control', 'coverage_experiment','coverage_control']]
tPCount = common_entries_total_peptide[common_entries_total_peptide['totalPeptides_experiment'] >=10]
common_entries_filtered= tPCount.sort_values(by='coverage_experiment',ascending=False)

saveInSheet(common_entries_filtered,part1SimilarFileName,part1Folder,'Primary Screening')
print(common_entries_filtered)
# saveInFolder(common_entries_total_peptide,'SimilarFS2.xlsx',part1Folder)


SimilarFS2.xlsx is saved.
            entry  totalPeptides_experiment  totalPeptides_control  \
387    GPX1_MOUSE                        12                      9   
70    K1C19_MOUSE                        29                     26   
10     TAGL_MOUSE                        20                     21   
1      ACTB_MOUSE                        22                     21   
190    CBR1_MOUSE                        14                     14   
...           ...                       ...                    ...   
1758   ITB4_MOUSE                        12                      6   
1938   GCN1_MOUSE                        18                      3   
1954  MYO5B_MOUSE                        12                      2   
1935  UGGG1_MOUSE                        11                      2   
1030   FLNC_MOUSE                        12                     12   

      coverage_experiment  coverage_control  
387                 70.15             48.76  
70                  66.75             53.

## Secondary Screening

In [66]:
# No MS2 and FS2 for >=5 
#
# # Modify the copy
common_entries_filtered['Ratio Mw/IpG'] = common_entries_filtered['totalPeptides_experiment'] / common_entries_filtered['totalPeptides_control']
common_entries_filtered= common_entries_filtered[['entry','Ratio Mw/IpG','totalPeptides_experiment','totalPeptides_control' ]]
s = common_entries_filtered[common_entries_filtered['Ratio Mw/IpG'] >=2 ]

# Create a Pandas Excel writer object
saveInSheet(s,part1SimilarFileName,part1Folder,'Secondary Screening 1')
# s = common_entries_filtered[common_entries_filtered['Ratio Mw/IpG'] >=5 ]
# saveInSheet(s,part1SimilarFileName,part1Folder,'Secondary Screening 2')


SimilarFS2.xlsx is saved.


## Part 2 

### Exclusive Male vs Exclusive Female

In [110]:
# folderpath , part1ExclusiveFileName, part1SimilarName
# SIMILAR_FILE= os.path.join(rootFolder,part1Folder,'Exclusive' )
EXCLUSIVE_FILE = os.path.join(rootFolder,part1Folder, 'Exclusive')

exclusiveMale = pd.read_excel(f'{EXCLUSIVE_FILE}ML2.xlsx')
exclusiveFemale = pd.read_excel(f'{EXCLUSIVE_FILE}FL2.xlsx')

print(exclusiveMale)
exclusiveFemale

         entry  coverage  totalPeptides
0  TBA1C_MOUSE     20.27             11
1  K1C17_MOUSE     19.17             16
2  K2C6A_MOUSE     16.64             11
3   SCEL_MOUSE     13.65             10
4  MTCL1_MOUSE     13.16             26
5  K1C14_MOUSE     13.02             12
6  FMNL1_MOUSE      8.23             11
7   WNK1_MOUSE      5.51             13
8  AKAP9_MOUSE      5.06             20
9  RN213_MOUSE      3.26             19


,entry,coverage,totalPeptides
0,ACTB_MOUSE,77.07,23
1,ACTG_MOUSE,77.07,23
2,CAZA2_MOUSE,73.78,13
3,ALDOA_MOUSE,73.35,17
4,CAH1_MOUSE,72.03,11
...,...,...,...
254,CO5_MOUSE,7.32,10
255,LAMB1_MOUSE,5.88,10
256,K0754_MOUSE,5.66,15
257,TLN2_MOUSE,4.93,11


In [111]:
onlyMale = exclusiveMale[~exclusiveMale['entry'].isin(exclusiveFemale['entry'])]
print(onlyMale)
onlyFemale = exclusiveFemale[~exclusiveFemale['entry'].isin(exclusiveMale['entry'])]
print(onlyFemale)
common_entries = pd.merge(onlyMale, onlyFemale, on='entry', suffixes=('_control', '_experiment'))
print(common_entries)
# Protein Present Male  [exlusive] |TC | Common |
# TC | Protein Present Present in Female only |TC
finalPd = pd.DataFrame({'Protein Present Male Only':onlyMale['entry'],'totalPeptide-M':onlyMale['totalPeptides'],'Common':common_entries['entry'],
                        'totalPeptides-Common':common_entries['totalPeptides_experiment'],
                        'Protein Present Present in Female only':onlyFemale['entry'],
                       'totalPeptide-F':onlyFemale['totalPeptides']})
finalPd
saveInFolder(finalPd,'FinalExclusiveL2.xlsx',part2Folder)

         entry  coverage  totalPeptides
1  K1C17_MOUSE     19.17             16
2  K2C6A_MOUSE     16.64             11
3   SCEL_MOUSE     13.65             10
4  MTCL1_MOUSE     13.16             26
5  K1C14_MOUSE     13.02             12
6  FMNL1_MOUSE      8.23             11
7   WNK1_MOUSE      5.51             13
8  AKAP9_MOUSE      5.06             20
           entry  coverage  totalPeptides
0     ACTB_MOUSE     77.07             23
1     ACTG_MOUSE     77.07             23
2    CAZA2_MOUSE     73.78             13
3    ALDOA_MOUSE     73.35             17
4     CAH1_MOUSE     72.03             11
..           ...       ...            ...
253   ANK1_MOUSE      8.27             10
254    CO5_MOUSE      7.32             10
255  LAMB1_MOUSE      5.88             10
256  K0754_MOUSE      5.66             15
257   TLN2_MOUSE      4.93             11

[257 rows x 3 columns]
Empty DataFrame
Columns: [entry, coverage_control, totalPeptides_control, coverage_experiment, totalPeptides_exp

### Similar Male vs Similar Female

In [5]:

SIMILAR_FILE= os.path.join(rootFolder,part1Folder,'Similar' )
similar_male = pd.read_excel(f'{SIMILAR_FILE}ML2.xlsx',sheet_name='Secondary Screening')
similar_female = pd.read_excel(f'{SIMILAR_FILE}FL2.xlsx',sheet_name='Secondary Screening')
print(similar_female)
print(similar_male)

           entry  Ratio Mw/IpG
0      HBA_MOUSE      2.000000
1     SBP1_MOUSE      3.100000
2    PRDX6_MOUSE      2.714286
3    K1C19_MOUSE      2.076923
4     HBB1_MOUSE      2.000000
..           ...           ...
217   ITB1_MOUSE      6.500000
218   CO4B_MOUSE      8.000000
219   PGBM_MOUSE     14.000000
220   PEPL_MOUSE     15.000000
221  SC31A_MOUSE     11.000000

[222 rows x 2 columns]
         entry  Ratio Mw/IpG
0   LMNA_MOUSE      3.142857
1   KINH_MOUSE      4.333333
2   DESP_MOUSE     11.333333
3  MYH14_MOUSE      3.666667


In [113]:
onlyMale = similar_male[~similar_male['entry'].isin(similar_female['entry'])]

onlyFemale = similar_female[~similar_female['entry'].isin(similar_male['entry'])]

common_m_f = pd.merge(onlyMale, onlyFemale, on='entry', suffixes=('_m', '_f'))

# Protein Present Male  [exlusive] |TC | Common |
# TC | Protein Present Present in Female only |TC
finalPd = pd.DataFrame({
    'Protein Present Male Only': onlyMale['entry'],
    'totalPeptide-M': onlyMale['totalPeptides_experiment'],
    'Common': common_m_f['entry'],
    'totalPeptides-Common': common_m_f['totalPeptides_experiment_m'],
    'Protein Present Present in Female only': onlyFemale['entry'],
    'totalPeptide-F': onlyFemale['totalPeptides_experiment']
})
df = finalPd.dropna(subset=['Protein Present Male Only','Protein Present Present in Female only'])

# After dropping rows with missing values, you might want to reset the index
finalPd = df.reset_index(drop=True)
finalPd
saveInFolder(finalPd,'FinalSimilarL2.xlsx',part2Folder)

FinalSimilarL2.xlsx is saved to: C:\Users\Jawwad\Desktop\Zaid Bhai Research\Dataset2 (Lung)\Part 2\FinalSimilarL2.xlsx


Notes:
1)Sheet 1 : Primary Screening : -
- Exclusive for difference Proteins Present in MWML2 | Total Peptide  :
- 
- Similar Experiment group TotalPeptide count >= 10: refine
- One file for  Present in both IgG/Mw | Ratio  
- file name  e.g : will be Similar ML2 and Exclusive ML2
- Coverage column needs to be there

2) Sheet 2 : Secondary Screening :
- male and female data comparasion . similar with similar and exlusive with exclusive .
- Protein Present Male  [exlusive] | Common | Protein Present Present in Female only 
- No totalPeptide
- Folder Name : Dataset 2 (lung) 
    - Part 1 
    - Part 2 
    - Final Screen Data 
- Dataset (Stomach)
- Dataset 2 (Stomach) 



## Secondary Screening 